# Modelo

In [28]:
import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error
import holidays
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from sklearn.preprocessing import MinMaxScaler

ModuleNotFoundError: No module named 'keras'

## Lectura de datos y feature engineeing

In [ ]:
Monterrey = pd.read_csv("Files/Datos_2022_2023_CENTRO_limpios.csv", parse_dates=[0], index_col = "date")
daily_Monterrey = Monterrey.resample('D').mean()

# Rango de años
years = range(2022, 2024)

# Crear una lista vacía para almacenar las fechas festivas
festivos = []

# Obtener los días festivos de México
mx_holidays = holidays.Mexico(years=years)

# Llenar la lista con los días festivos y sus nombres
for date, name in mx_holidays.items():
    festivos.append([date, name])

# Crear un DataFrame con las fechas y nombres de los días festivos
festivos_df = pd.DataFrame(festivos, columns=['Fecha', 'Evento'])

# Convertir a formato de fecha
festivos_df['Fecha'] = pd.to_datetime(festivos_df['Fecha'])

daily_Monterrey['is_holiday'] = daily_Monterrey.index.isin(festivos_df['Fecha']).astype(int)

# Crear la variable dummy para fines de semana
daily_Monterrey['is_weekend'] = daily_Monterrey.index.weekday.isin([5, 6]).astype(int)


In [ ]:
daily_Monterrey.shape

(594, 17)

## Escalamiento 

In [ ]:
# Inicializamos el escalador
scaler = MinMaxScaler()

# Escalamos todos los datos
scaled_data = scaler.fit_transform(daily_Monterrey)

NameError: name 'MinMaxScaler' is not defined

## Entrenamiento, validación y testeo

In [ ]:
# Número de días para testeo y validación
n_test = 31
n_val = 31

# Últimos 31 datos para testeo
X_test = daily_Monterrey.iloc[-n_test:]["CO"]
y_test = daily_Monterrey.iloc[-n_test:].drop(columns=["CO"])

# 31 anteriores a los de testeo para validación
X_val = daily_Monterrey.iloc[-(n_test + n_val):-n_test]["CO"]
y_val = daily_Monterrey.iloc[-(n_test + n_val):-n_test].drop(columns=["CO"])

# El resto (excepto los últimos 62) para entrenamiento
X_train = daily_Monterrey.iloc[:-(n_test + n_val)]["CO"]
y_train = daily_Monterrey.iloc[:-(n_test + n_val)].drop(columns=["CO"])